In [1]:
import pandas as pd 
import numpy as np 
import uuid

This function is responsible for generating (export) excel files

In [2]:
def export_to_excel(df, file_name='output.xlsx', index=False):
    """
    Exports a DataFrame to an Excel file.

    Parameters:
    - df (pd.DataFrame): The DataFrame to export.
    - file_name (str): Name of the output Excel file.
    - sheet_name (str): Name of the sheet in Excel.
    - index (bool): Whether to include the DataFrame index.
    """
    try:
        df.to_excel(f'tables/{file_name}',  index=index)
        print(f"✅ Data exported successfully to '{file_name}'")
    except Exception as e:
        print(f"❌ Failed to export: {e}")

In [3]:


df = pd.read_excel("fact_downtime_.xlsx")


In [4]:
df = df.dropna()



We have a data table that includes columns corresponding to dimension tables.  
The goal is to:

- Generate unique IDs using the `uuid` package.
- Separate these columns into individual **dimension tables**.
- Construct a **fact table** containing:
  - The necessary features.
  - The IDs of the associated dimension tables.

**Example:**  
The column `produit` would be processed by:
- Generating a `product_id`.
- Renaming `produit` to `product_name`.
- Moving this information into a `product` dimension table.


#### **dim manifact event**

In [5]:
# Replacing 'Bosch' with 'BOSCH' in the 'Machinist' column
df['Machinist'] = df['Machinist'].apply(lambda x: x.replace('Bosch', 'BOSCH') if isinstance(x, str) else x)


In [6]:
df['manufacturing_event_code'] = df['Machinist'].apply(lambda x: uuid.uuid5(uuid.NAMESPACE_DNS, str(x)))

In [7]:
df.rename(columns={'Machinist': 'event_desc'}, inplace=True)

In [8]:
dim_mani_event = df[['manufacturing_event_code','event_desc' ]].drop_duplicates().dropna()

In [9]:
export_to_excel(dim_mani_event, 'dim_manifact_event.xlsx')

✅ Data exported successfully to 'dim_manifact_event.xlsx'


#### **dim downtime**

In [10]:
df['downtime_id'] = df['Type'].apply(lambda x: uuid.uuid5(uuid.NAMESPACE_DNS, str(x)) if pd.notnull(x) else None)

In [11]:
df.rename(columns={'Type': 'downtime_reason'}, inplace=True)

In [12]:
dim_downtime = df[['downtime_id', 'downtime_reason']].drop_duplicates().dropna()

In [13]:
export_to_excel(dim_downtime, 'dim_downtime.xlsx')

✅ Data exported successfully to 'dim_downtime.xlsx'


#### **dim event details**

In [14]:
# manifact_event_details
df.rename(columns={'ARRET ': 'manifact_event_details'}, inplace=True)

In [15]:
df['event_details_id'] = df['manifact_event_details'].apply(lambda x: uuid.uuid5(uuid.NAMESPACE_DNS, str(x)) if pd.notnull(x) else None)

In [16]:
dim_event_details = df[['event_details_id','manifact_event_details']].drop_duplicates().dropna()

In [17]:
export_to_excel(dim_event_details, 'dim_event_details.xlsx')

✅ Data exported successfully to 'dim_event_details.xlsx'


#### **dim wate declaration** 

In [18]:
## chargement fact waste data
df1 = pd.read_excel("fact_waste_.xlsx")


In [19]:
df1 = df1.dropna()

In [20]:
df1['Déclaration Déchet'] = df1['Déclaration Déchet'].apply(
    lambda x: x.strip().replace(':', '').upper() if isinstance(x, str) else x
)


In [21]:
df1['Déclaration Déchet'] = df1['Déclaration Déchet'].fillna('UNKNOWN')


In [22]:
df1.rename(columns={'Déclaration Déchet': 'wate_declaration'}, inplace=True)

In [23]:
df1['wate_declaration_id'] = df1['wate_declaration'].apply(lambda x: uuid.uuid5(uuid.NAMESPACE_DNS, str(x)) if pd.notnull(x) else None)

In [24]:
dim_wate_dec = df1[['wate_declaration_id', 'wate_declaration']].drop_duplicates().dropna()

In [25]:
export_to_excel(dim_wate_dec, 'dim_wate_declare.xlsx')

✅ Data exported successfully to 'dim_wate_declare.xlsx'


### **dim product**

In [26]:
def split_product_name_id(val):
    if pd.isnull(val):
        return pd.Series([np.nan, np.nan])
    parts = val.strip().rsplit(' ', 1)
    if len(parts) == 2:
        return pd.Series([parts[0].strip(), parts[1].strip()])
    else:
        return pd.Series([val.strip(), np.nan])

# Apply function
df[['product_name', 'product_id']] = df['Produit'].apply(split_product_name_id)

In [27]:
df1[['product_name', 'product_id']] = df1['Produit'].apply(split_product_name_id)

In [28]:
## chargement fact production performance
df2 = pd.read_excel("fact_prod_.xlsx")

In [29]:
df2 = df2.dropna(subset=['Date'])

In [30]:
df2 = df2[df2['Produit'] != 120.192307692308]

In [31]:
# df2[['product_name', 'product_id']] = df2['Produit'].apply(split_product_name_id)
df2[['product_name', 'product_id']] = df1[['product_name', 'product_id']]

In [32]:
dim_product = df1[['product_name', 'product_id']].drop_duplicates().dropna()

In [33]:
export_to_excel(dim_product, 'dim_product.xlsx')

✅ Data exported successfully to 'dim_product.xlsx'


### **dim leader**

In [34]:
df.rename(columns={'Chef d\'equipe': 'leader_name'}, inplace=True)

In [35]:
df['leader_id'] = df['leader_name'].apply(lambda x: uuid.uuid5(uuid.NAMESPACE_DNS, str(x)) if pd.notnull(x) else None)

In [36]:
df1.rename(columns={'Chef d\'equipe': 'leader_name'}, inplace=True)

In [37]:
df1['leader_id'] = df1['leader_name'].apply(lambda x: uuid.uuid5(uuid.NAMESPACE_DNS, str(x)) if pd.notnull(x) else None)

In [38]:
df2.rename(columns={'Chef d\'equipe': 'leader_name'}, inplace=True)

In [39]:
df2['leader_id'] = df2['leader_name'].apply(lambda x: uuid.uuid5(uuid.NAMESPACE_DNS, str(x)) if pd.notnull(x) else None)

In [40]:
dim_leader = df[['leader_id', 'leader_name']].drop_duplicates().dropna()

In [41]:
export_to_excel(dim_leader, 'dim_leader.xlsx')

✅ Data exported successfully to 'dim_leader.xlsx'


### **dim group**

In [42]:
df.rename(columns={'N° Groupe ': 'group_num'}, inplace=True)
df1.rename(columns={'N° Groupe ': 'group_num'}, inplace=True)
df2.rename(columns={'N° Groupe ': 'group_num'}, inplace=True)

In [43]:
dim_group = df['group_num'].drop_duplicates().dropna()

In [44]:
export_to_excel(dim_group, 'dim_group.xlsx')

✅ Data exported successfully to 'dim_group.xlsx'


### **dim date**

In [45]:
dim_date = df[['Date', 'N° SEMAINE']].drop_duplicates().dropna()

In [46]:
dim_date = dim_date.dropna()
dim_date = dim_date.drop(columns='N° SEMAINE')

In [47]:
dim_date = dim_date.drop_duplicates()

In [48]:
dim_date.reset_index(drop=True, inplace=True)


In [49]:
export_to_excel(dim_date, 'dim_date1.xlsx')

✅ Data exported successfully to 'dim_date1.xlsx'


### **Fact_ProductionPerformance**

In [50]:
columns_to_select = [
    '(+ / -) Objectife',        # Column related to Objectif
    'Déchet  Mnt)',             # Column related to Waste
    'Pourcentage Déchet',
    'Pourcentage Arret',
    'Pourcentage PDP',
    'Pourcentage Pf',
    ' PF  %',
    'Temps arret',              # Column related to Downtime
    'Temps arret PRG',          # Column related to Programmed Downtime
    'Cadence = Four',           # Column related to Cadence
    'Date',                     # Date column
    'group_num',             #n ggroupe
    'Objectife',                # Objectif column
    'PDP',                      # PDP column
    'Production',               # Production column
    'Temps Service PF',         # Service time PF column
    'leader_id',              # Teamleader column
    'TRS',                      # TRS column
    'product_id'                # Product ID column
]

fact_production = df2[columns_to_select]
fact_production.head()


,(+ / -) Objectife,Déchet Mnt),Pourcentage Déchet,Pourcentage Arret,Pourcentage PDP,Pourcentage Pf,PF %,Temps arret,Temps arret PRG,Cadence = Four,Date,group_num,Objectife,PDP,Production,Temps Service PF,leader_id,TRS,product_id
0,-768.0,10.774411,0.022447,0.479167,0.573333,0.501613,0.477778,230.0,10.0,4.5,2025-01-06,2 - A,1800.0,1800.0,1032,229.333333,9fb5796f-3013-5b23-998b-64be172e644a,2160,PFBS16K00054
1,-188.0,19.865320,0.041386,0.191667,0.895556,0.233053,0.746296,92.0,10.0,4.5,2025-01-06,3 - B,1800.0,1800.0,1612,358.222222,32d8b501-5b38-5767-b8b9-43b50754b34f,2160,PFBS16K00054
2,6.0,15.656566,0.032618,0.110417,1.003333,0.143035,0.836111,53.0,10.0,4.5,2025-01-07,1 - C,1800.0,1800.0,1806,401.333333,c59267b5-cd00-5b22-9bcf-814b089b95e2,2160,PFBS16K00054
3,-451.0,18.181818,0.037879,0.316667,0.167917,0.354545,0.624537,152.0,10.0,4.5,2025-01-07,2 - A,1800.0,7200.0,1209,299.777778,e93f01c3-5a4c-584b-a3ca-5fec227000f5,2160,PFBS16K00054
4,-555.0,10.606061,0.022096,0.381250,0.691667,0.403346,0.576389,183.0,10.0,4.5,2025-01-07,3 - B,1800.0,1800.0,1245,276.666667,9fb5796f-3013-5b23-998b-64be172e644a,2160,PFBS16K00054


In [51]:
export_to_excel(fact_production, 'fact_production.xlsx')

✅ Data exported successfully to 'fact_production.xlsx'


### **Fact Waste**

In [52]:

columns_to_select_waste = [
    'Date',                 # Date column
    'Dechet %',             # Dechet % column
    'Kg',                   # Kg column
    'group_num',            # N° Groupe column
    'product_id',              # Produit column
    'leader_id' ,       # Chef d'équipe column (using leader_id)
    'wate_declaration_id',     # Déclaration Déchet column
]


df_fact_waste = df1[columns_to_select_waste]
df_fact_waste.head()


,Date,Dechet %,Kg,group_num,product_id,leader_id,wate_declaration_id
0,2025-01-06,0.000547,8,2 - A,PFBS16K00054,9fb5796f-3013-5b23-998b-64be172e644a,00001151-cf1b-580f-bb3d-ec6a91b8528e
1,2025-01-06,0.013687,200,2 - A,PFBS16K00054,9fb5796f-3013-5b23-998b-64be172e644a,9623a13d-aedf-5d55-9df3-b405f5e860f9
2,2025-01-06,0.005475,80,2 - A,PFBS16K00054,9fb5796f-3013-5b23-998b-64be172e644a,476f6d60-5f97-54ab-b6c9-87a1217bd8d7
3,2025-01-06,0.002737,40,2 - A,PFBS16K00054,9fb5796f-3013-5b23-998b-64be172e644a,f4dda7fb-97c7-503d-af34-de0b8f015757
4,2025-01-06,0.000622,9,3 - B,PFBS16K00054,32d8b501-5b38-5767-b8b9-43b50754b34f,00001151-cf1b-580f-bb3d-ec6a91b8528e


In [53]:
export_to_excel(df_fact_waste, 'fact_waste.xlsx')

✅ Data exported successfully to 'fact_waste.xlsx'


### **Fact downtime**

In [54]:
columns_to_select_downtime = [
    'Date',                  # Date column
    'Déclaration arret',     # Downtime declaration (Déclaration arret)
    'group_num',             # Groupe Name (group_num)
    'leader_id',           # Machinist (leader_name)
    'product_id',               # Product (Produit)                 
    'manufacturing_event_code'             # Type (event_desc)
]


df_fact_downtime = df[columns_to_select_downtime]


df_fact_downtime.head()


,Date,Déclaration arret,group_num,leader_id,product_id,manufacturing_event_code
0,2025-01-06,103 Préparation de la ligne au démarrage,2 - A,9fb5796f-3013-5b23-998b-64be172e644a,PFBS16K00054,b6819935-b4e2-55b7-a0da-2c988ff1fa54
1,2025-01-06,521 Four mixte FIB 1.5×52,2 - A,9fb5796f-3013-5b23-998b-64be172e644a,PFBS16K00054,0056e7ba-998c-5e0a-af87-177bdaa61d04
2,2025-01-06,526 Classeur de biscuit,2 - A,9fb5796f-3013-5b23-998b-64be172e644a,PFBS16K00054,756b4c0f-c6e3-572c-aa1f-5f62b6425ce6
3,2025-01-06,511 Arrêt technique enveloppeuse,2 - A,9fb5796f-3013-5b23-998b-64be172e644a,PFBS16K00054,81b7a44c-51fa-5cc9-a5b1-d7ba169dea69
4,2025-01-06,103 Préparation de la ligne au démarrage,3 - B,32d8b501-5b38-5767-b8b9-43b50754b34f,PFBS16K00054,b6819935-b4e2-55b7-a0da-2c988ff1fa54


In [55]:
export_to_excel(df_fact_downtime, 'fact_downtime.xlsx')

✅ Data exported successfully to 'fact_downtime.xlsx'


### **exporting tables**

In [56]:
export_to_excel(df, 'fact_downtime_.xlsx')
export_to_excel(df1, 'fact_waste_.xlsx')
export_to_excel(df2, 'fact_prod_.xlsx')

✅ Data exported successfully to 'fact_downtime_.xlsx'
✅ Data exported successfully to 'fact_waste_.xlsx'
✅ Data exported successfully to 'fact_prod_.xlsx'
